# Collect game information

We will be using information from http://survivor.wikia.com

## Dependencies

In [1]:
from bs4 import BeautifulSoup
import urllib2
import re

import numpy as np
import pandas as pd
import itertools as iter
from IPython import display as dis
import pickle

In [2]:
# silly utility to launch a qtconsole if one doesn't exist
import psutil

def returnPyIDs():
    pyids = set()
    for pid in psutil.pids():
        try:
            if "python" in psutil.Process(pid).name():
                pyids.add(pid)
        except:
            pass
    return pyids

def launchConsole():
    before_pyids = returnPyIDs()
    %qtconsole
    after_pyids = returnPyIDs()
    newid = after_pyids.difference(before_pyids)
    assert len(newid) == 1
    return list(newid)[0]

try:
    qtid
except NameError:
    qtid = launchConsole()
    
if qtid not in returnPyIDs():
    qtid = launchConsole()
    
qtid

1504

## Parsing Functions

In [3]:
def parse_td(td):
    # Extract text from table data element
    try:
        # if there is a colspan element present ...
        n = int(td['colspan'])
    except KeyError:
        n = 1
    # (these all end with \n so strip that)
    return [td.text[:-1]] * n  # ... repeat text n times

def get_voting_results(url):
    html= urllib2.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    voting_table = soup.find(id='Voting_History').find_next('table')
    
    # Eliminated
    # find the 'tr' flag right before the 'th' flag with text described by reg. exp.
    # (use iter.chain to flatten list)
    eliminated = list(iter.chain(*[parse_td(i) for i in 
              voting_table
              .find('th', text=re.compile(r"(Voted Out|Eliminated|Voted Off)"))
              .findPrevious('tr').findAll('td')]))
    
    # we want all rows after the row that starts with "Vote:"
    voting_rows = voting_table.find('th', text=re.compile(r"Vote:")).findAllNext('tr')
    # extract the text from all table data in the rows we just extracted
    votes = [list(iter.chain(*[parse_td(i) for i in j.findAll('td')])) for j in voting_rows]

    # convert to pandas data frame
    votes = pd.DataFrame(votes)
    # the contestant casting the vote is in the second row - make that the index
    votes.index = votes[1]
    # now we can drop the first two columns
    votes.drop(votes.columns[[0, 1]], inplace=True, axis=1)
    # drop any rows where the index is None
    votes.drop(votes.index[[i == None for i in votes.index]], inplace=True, axis=0)
    # Our columns correspond to those eliminated (from above)
    # There appears to be an extra column. Drop anthing beyoned the elimination info we have
    votes = votes[votes.columns[[range(len(eliminated))]]]
    # rename columns to be the eliminated contestants
    votes.columns = eliminated
    return votes

def get_season_info(url="http://survivor.wikia.com/wiki/Main_Page"):
    html= urllib2.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    # collect all link references that begin with /wiki/Survivor:
    season_refs = set(
        [i['href'] for i in soup.findAll('a', {'href': re.compile("^/wiki/Survivor:")})]
    )
    # we get the season url by adding the following prefix
    prefix = "http://survivor.wikia.com"
    season_urls = [prefix + i for i in list(season_refs)]
    # create a dictionary of this info
    seasons = {}
    for url in season_urls:
        name = url.split(':_')[-1]
        seasons[name] = {}
        seasons[name]['url'] = url
        
    return seasons
    

## Scrape All seasons

Scrape info

In [4]:
url = "http://survivor.wikia.com/wiki/Main_Page"
seasons = get_season_info(url)

for i in seasons.keys():
    season = seasons[i]  # A dictionary
    season['votes'] = get_voting_results(season['url'])

Save Info

In [5]:
pickle.dump( seasons, open( "wiki_scrape.p", "wb" ) )

## Sample season results

In [6]:
get_voting_results(seasons['San_Juan_del_Sur']['url'])

,Nadiya,Val,Val,John,Drew,Kelley,Dale,Julie,Josh,Jeremy,Wes,Reed,Alec,Jon,Jon,Baylor,Keith,Missy,Jaclyn,Natalie
1,,,,,,,,,,,,,,,,,,,,
Natalie,—,—,—,—,Drew,—,—,—,Josh,Keith,Keith,Reed,Alec,Jon,Jon,Baylor,Keith,Jury Vote,Jury Vote,Jury Vote
Jaclyn,Dale,Baylor,Baylor,John,—,Kelley,Dale,—,Josh,Jeremy,Wes,Reed,Keith,Keith,None,Baylor,Keith,None,None,None
Missy,—,—,—,—,Drew,Kelley,Keith,—,Josh,Jeremy,Keith,Reed,Alec,Jaclyn,Jaclyn,Jaclyn,Keith,None,None,None
Keith,—,—,—,—,Julie,Dale,Dale,—,Baylor,Reed,Jon,Reed,Alec,Jon,None,Jaclyn,Missy,,,Natalie
Baylor,Nadiya,Val,None,John,—,Kelley,Keith,—,Josh,Jeremy,Keith,Reed,Alec,Jaclyn,Jon,Jaclyn,,Missy,,
Jon,—,—,—,—,Keith,Dale,Dale,—,Josh,Jeremy,Wes,Reed,Keith,Keith,None,,,,Jaclyn,
Alec,Nadiya,Baylor,Val,John,—,—,—,—,Baylor,Reed,Jon,Reed,Keith,,,,,,,Natalie
Reed,—,—,—,—,Julie,—,—,—,Baylor,Jeremy,Jon,Missy,,,,,,,Jaclyn,
Wes,Nadiya,Val,Val,John,—,—,—,—,Baylor,Reed,Jon,,,,,,,,,Natalie


## Display All results

In [7]:
for k in seasons.keys():
    season = seasons[k]
    print "{}: {}".format(k, season['url'])
    dis.display(season['votes'])

Palau: http://survivor.wikia.com/wiki/Survivor:_Palau


,Jonathan,Wanda,Jolanda,Ashlee,Jeff,Kim,Willard,Angie,Angie,James,...,Coby,Janu,Stephenie,Gregg,Caryn,Jenn,Jenn,Ian,Katie,Tom
1,,,,,,,,,,,,,,,,,,,,,
Tom,—,—,—,—,—,—,Willard,—,—,—,...,Coby,—,Stephenie,Gregg,Caryn,Ian,Ian,Ian,None,None
Katie,—,—,—,—,—,—,Willard,—,—,—,...,Coby,—,Stephenie,Gregg,Caryn,Jenn,Jenn,—,None,None
Ian,—,—,—,—,—,—,Willard,—,—,—,...,Coby,—,Stephenie,Gregg,Caryn,Jenn,None,—,,Tom
Jenn,—,—,—,—,—,—,Willard,—,—,—,...,Coby,—,Stephenie,Caryn,Caryn,Ian,None,,,Tom
Caryn,—,—,—,—,—,—,Willard,—,—,—,...,Coby,—,Stephenie,Gregg,Ian,,,,,Tom
Gregg,—,—,—,—,—,—,Willard,—,—,—,...,Coby,—,Stephenie,Caryn,,,,,,Tom
Stephenie,—,—,Jolanda,Ashlee,Jeff,Kim,—,Bobby Jon,Angie,Ibrehem,...,Coby,—,Caryn,,,,,,,Tom
Janu,—,—,—,—,—,—,Willard,—,—,—,...,Stephenie,Quit,,,,,,,,Tom
Coby,—,—,—,—,—,—,Willard,—,—,—,...,Janu,,,,,,,,Katie,


Tocantins: http://survivor.wikia.com/wiki/Survivor:_Tocantins


,Carolina,Candace,Jerry,Sandy,Spencer,Sydney,Joe,Brendan,Tyson,Sierra,Debbie,Coach,Taj,Erinn,Stephen,J.T.
1,,,,,,,,,,,,,,,,
J.T.,Carolina,—,—,Sandy,Spencer,Sydney,—,Sierra,Tyson,Sierra,Debbie,Erinn,Taj,Erinn,None,None
Stephen,Carolina,—,—,Sandy,Spencer,Sydney,—,Brendan,Tyson,Sierra,Debbie,Coach,Taj,—,None,None
Erinn,—,Candace,Jerry,—,—,—,—,Sierra,Tyson,Stephen,Debbie,Coach,Taj,—,,J.T.
Taj,Carolina,—,—,Joe,Spencer,Sydney,—,Brendan,Tyson,Debbie,Debbie,Coach,Erinn,,,J.T.
Coach,—,Candace,Jerry,—,—,—,—,Brendan,Sierra,Sierra,Taj,Erinn,,,,J.T.
Debbie,—,Candace,Jerry,—,—,—,—,Brendan,Sierra,Sierra,Coach,,,,,J.T.
Sierra,—,Candace,Jerry,—,—,—,—,Coach,Tyson,Debbie,,,,,,J.T.
Tyson,—,Candace,Jerry,—,—,—,—,Sierra,Sierra,,,,,,,J.T.
Brendan,—,Candace,Jerry,—,—,—,—,Coach,,,,,,,,J.T.


Borneo: http://survivor.wikia.com/wiki/Survivor:_Borneo


...

,Sekou,Billy,Cecilia,J.P.,Stephannie,Cao Boi,Cristina,Jessica,Brad,Rebecca,Jenny,Nate,Candice,Jonathan,Parvati,Adam,Sundra,Becky,Ozzy,Yul
1,,,,,,,,,,,,,,,,,,,,
Yul,—,—,Cecilia,—,—,Cao Boi,—,Jessica,—,—,—,Nate,Candice,Jonathan,Parvati,Adam,Sundra,None,None,None
Ozzy,—,Billy,Becky,—,—,Cao Boi,—,Jessica,—,—,—,Nate,Candice,Jonathan,Parvati,Adam,Becky,None,None,None
Becky,—,—,Cecilia,—,—,Cao Boi,—,Jessica,—,—,—,Nate,Candice,Jonathan,Parvati,Adam,Sundra,None,None,None
Sundra,Sekou,—,Becky,—,—,Cao Boi,—,Jessica,—,—,—,Nate,Candice,Jonathan,Parvati,Adam,Becky,,,Yul
Adam,—,—,—,J.P.,Stephannie,—,Cristina,—,Brad,Rebecca,Jenny,Yul,Jonathan,Jonathan,Sundra,Yul,,,,Yul
Parvati,—,—,—,J.P.,Stephannie,—,Cristina,—,Brad,Rebecca,Jenny,Yul,Jonathan,Jonathan,Sundra,,,,Ozzy,
Jonathan,—,—,Cecilia,—,—,Cao Boi,—,Jessica,Brad,Rebecca,Jenny,Nate,Candice,Adam,,,,,,Yul
Candice,—,—,Exiled,—,—,Cao Boi,—,Jessica,Brad,Rebecca,Jenny,Yul,Jonathan,,,,,,,Yul
Nate,Sundra,—,—,Stephannie,Stephannie,—,Kidnapped,—,Brad,Rebecca,Jonathan,Yul,,,,,,,Ozzy,


Redemption_Island: http://survivor.wikia.com/wiki/Survivor:_Redemption_Island


,Francesca,Matthew,Russell,Russell,Kristina,Krista,Stephanie,Sarita,Matthew,Mike,...,Julie,Ralph,Steve,Andrea,Grant,Andrea,Ashley,Natalie,Phillip,Rob
1,,,,,,,,,,,,,,,,,,,,,
Rob,Francesca,Matthew,—,—,Phillip,—,—,—,Matthew,Mike,...,Julie,Ralph,Steve,Andrea,Grant,Andrea,Ashley,None,None,None
Phillip,Francesca,Kristina,—,—,Kristina,—,—,—,Matthew,Mike,...,Julie,Ralph,Steve,Andrea,Grant,Andrea,Ashley,None,None,None
Natalie,Kristina,Matthew,—,—,Kristina,—,—,—,Matthew,Mike,...,Julie,Ralph,Steve,Andrea,Grant,Andrea,Ashley,None,None,None
Ashley,Kristina,Matthew,—,—,Kristina,—,—,—,Matthew,Mike,...,Julie,Ralph,Steve,Andrea,Grant,Andrea,Phillip,,,Rob
Andrea,Kristina,Kristina,—,—,Kristina,—,—,—,Matthew,Mike,...,Julie,Ralph,Steve,Phillip,,Rob,,,,Rob
Mike,—,—,Russell,Russell,—,Krista,Stephanie,Sarita,Grant,Phillip,...,,,,,,,,,,Rob
Matthew,Francesca,Phillip,,,,,,,Steve,,...,,,,,,,,,,Rob
Grant,Francesca,Matthew,—,—,Phillip,—,—,—,Matthew,Mike,...,Julie,Ralph,Steve,Andrea,Natalie,,,,,Rob
Ralph,—,—,Russell,None,—,Krista,Stephanie,David,Grant,Phillip,...,Phillip,Rob,,,,,,,Phillip,


Nicaragua: http://survivor.wikia.com/wiki/Survivor:_Nicaragua


,Wendy,Shannon,Jimmy J.,Jimmy T.,Tyrone,Kelly B.,Kelly B.,Yve,Jill,Alina,...,Brenda,NaOnka,Kelly S.,Benry,Jane,Dan,Holly,Sash,Chase,Fabio
1,,,,,,,,,,,,,,,,,,,,,
Fabio,—,Brenda,—,—,—,Kelly B.,Kelly B.,—,Marty,Alina,...,Brenda,—,—,Holly,Jane,Dan,Holly,None,None,None
Chase,—,Shannon,—,—,Tyrone,—,—,Yve,—,Alina,...,Brenda,—,—,Benry,Jane,Dan,Holly,None,None,None
Sash,—,Shannon,—,—,—,Marty,Kelly B.,—,Jill,Alina,...,Brenda,—,—,Benry,Jane,Dan,Holly,None,None,None
Holly,Wendy,—,Jimmy J.,Dan,Tyrone,—,—,Yve,—,Alina,...,Brenda,—,—,Benry,Jane,Dan,Sash,,Chase,
Dan,Wendy,—,Jimmy J.,Jimmy T.,Tyrone,—,—,Yve,—,Alina,...,Brenda,—,—,Fabio,Jane,Chase,,,,Fabio
Jane,Wendy,—,Jimmy J.,Dan,—,Marty,Marty,—,Marty,Marty,...,Brenda,—,—,Benry,Sash,,,,Chase,
Benry,—,Shannon,—,—,Tyrone,—,—,Yve,—,Alina,...,Brenda,—,—,Fabio,,,,,,Fabio
Kelly S.,—,Shannon,—,—,—,Kelly B.,Kelly B.,—,Jill,Alina,...,Benry,Quit,Quit,,,,,,,Fabio
NaOnka,—,Shannon,—,—,Tyrone,—,—,Yve,—,Alina,...,Brenda,Quit,Quit,,,,,,,Fabio
